In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl

from smot.common.runtime import build_paths, notebooks

experiment_log_dir = notebooks.experiment_log_dir()

import tensorboard

ImportError: cannot import name 'UnencryptedCookieSessionFactoryConfig' from 'pyramid.session' (unknown location)

In [ ]:
%load_ext tensorboard

In [ ]:
#  %tensorboard --logdir $experiment_log_dir
tensorboard.notebook.start(f'--logdir={experiment_log_dir}')

In [ ]:
tensorboard.notebook.display()

In [ ]:
class LitAutoEncoder(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28)
        )

    def forward(self, x):
        embedding = self.encoder(x)
        return embedding

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.log("val_loss", loss)


# data
dataset = MNIST(
    root=build_paths.data_root(),
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)
mnist_train, mnist_val = random_split(dataset, [55000, 5000])

train_loader = DataLoader(
    mnist_train,
    batch_size=32,
    num_workers=32,
)
val_loader = DataLoader(
    mnist_val,
    batch_size=32,
    num_workers=32,
)

# model
model = LitAutoEncoder()

# training
trainer = pl.Trainer(
    gpus=-1,
    strategy="dp",
    amp_backend="apex",
    num_nodes=1,
    precision=16,
    limit_train_batches=0.5,
    logger=True,
    stochastic_weight_avg=True,
    gradient_clip_algorithm="norm",
    default_root_dir=experiment_log_dir,
    auto_lr_find=True,
    auto_select_gpus=True,
    auto_scale_batch_size=True,
    max_epochs=20,
    # enable_progress_bar=True,
    enable_model_summary=True,
)
trainer.fit(model, train_loader, val_loader)